# Задание 1 (4 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

## Решение:

*NOTE: я не знаю, как умудрился получить такие результаты. Прошу прощения за полотна дублированного кода.*

### Загрузка датасета


In [44]:
import pathlib

corpus_path = pathlib.Path(r"data/corpus_wsd_50k.txt")

with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.read().split('\n\n')
    corpus = map(
        lambda sent: [
            word.split("\t")
            for word
            in sent.split("\n")
            ],
        corpus
        )
    corpus = tuple(corpus)

###  Загрузка ворднета

In [45]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
    

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kirill\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


###  Функция, составляющая контекст

Конетекстом считаем предложение, в котором встречается слово, без самого слова. Будем брать предложение из датасета целиком и маскировать в нем слово, для которого нужно определить значение. 

In [46]:
def get_context_mask_target(corpus_sentence:list, target_ind:int) -> list:

    result = list()

    for ind, word in enumerate(corpus_sentence):
        if ind != target_ind:
            result.append(word[1])
        else:
            result.append("_")

    return result

### Препроц

In [47]:
from string import punctuation


class Preprocessor:

    stopwords:list[str] = nltk.corpus.stopwords.words("english")
    stopwords.extend(punctuation)
    stopwords.append("_")
    stopwords = set(stopwords)

    @classmethod
    def tokenize(cls, text:str) -> list[str]:
        return nltk.word_tokenize(text)
    
    @classmethod
    def rm_stopwords(cls, tokenized_text:list[str]) -> list[str]:
        return [c for c in tokenized_text if c not in cls.stopwords]

    @classmethod
    def preprocess(cls, text: str | list[str]) -> list[str]:
        
        if isinstance(text, str):
            text = cls.tokenize(text)
        
        text = [c.lower() for c in text]      
        text = cls.rm_stopwords(text)

        return text

In [48]:
#  Проверяем препроц

s = corpus[1]
s = get_context_mask_target(s, 8)
s = Preprocessor.preprocess(s)
s

['permit',
 'become',
 'giveaway',
 'rather',
 'one',
 'goal',
 'improved',
 'employee',
 'morale',
 'consequently',
 'increased',
 'productivity']

###  Алгоритм Леска в общем виде

In [255]:
class Lesk:

    #  Псевдометрика, переопределяется в классах-наследниках
    metric = lambda sent1, sent2: 0

    @classmethod
    def _get_all_definitions(cls, word:str) -> str:
        return [c.definition() for c in wn.synsets(word)]


    @classmethod
    def get_definition(cls, word:str, context:list[str]) -> str:

        all_results = [
            {
                "definition": definition,
                "score"     : cls.metric(context, Preprocessor.preprocess(definition))
            }
            for definition
            in cls._get_all_definitions(word)
        ]

        return max(all_results, key=lambda c: c["score"])


### Алгоритм Леска с метрикой Жаккара

In [256]:
def jaccard(sent1:list[str], sent2:list[str]) -> float:
    sent1 = set(sent1)
    sent2 = set(sent2)

    return len(sent1 & sent2) / len(sent1 | sent2)


class LeskWithJaccard(Lesk):
    metric = jaccard

In [257]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'score': 0.3}

In [258]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithJaccard.get_definition(word, ctx)

{'definition': 'a motor vehicle with four wheels; usually propelled by an internal combustion engine',
 'score': 0.0}

### Алгоритм Леска с косинусной близостью на эмбедингах sentence_transformers

In [184]:
from sklearn.metrics.pairwise import cosine_distances
from sentence_transformers import SentenceTransformer

In [185]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [259]:

def cos_sim(sent1:list[str], sent2:list[str]) -> float:
    emb1 = model.encode(" ".join(sent1)).reshape(1, -1)
    emb2 = model.encode(" ".join(sent2)).reshape(1, -1)

    return float(cosine_distances(emb1, emb2)[0][0])

class LeskWithCosSim(Lesk):
    metric = cos_sim

In [260]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'where passengers ride up and down',
 'score': 0.8515400886535645}

In [261]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = Preprocessor.preprocess(ctx)

LeskWithCosSim.get_definition(word, ctx)

{'definition': 'a wheeled vehicle adapted to the rails of railroad',
 'score': 0.928614616394043}

### Всё готово для сравнительного тестирования

Осталось пройти по всему датасету, предсказать значения для всех многозначных слов обоими способами и сравнить результаты с эталонными.

In [209]:
from tqdm import tqdm

In [132]:
results = dict()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():
            continue

        else:

            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                context
            )

            cos_sim_result = LeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [04:08<00:00, 24.81s/it]


Первые 10 предложений в корпусе обрабатываются 4 с лишним минуты. Что вдвойне обидно, если принять во внимание тот факт, что в коде выше я забыл записать результаты замеров. Тем не менее, есть подозрение, что функция расчета косинусного подобия работает слишком медленно. Ниже попробую оптимизировать ее:

In [288]:

import numpy as np

##  Тут поменяем интерфейс так, чтобы не было лишних операций str.join
##  и заменим функцию расчета косинусной близости, чтобы модель кодировала контекст только 1 раз.
def better_cos_sim(context_sent:str, other_sents:list[str]) -> np.ndarray:
    context_emb = model.encode(context_sent).reshape(1, -1)
    other_embs = [
        model.encode(sent)
        for sent
        in other_sents
    ]
    return cosine_distances(context_emb, other_embs)[0]


##  Тут поменяем логику под новый расчет метрики
class BetterLeskWithCosSim(Lesk):
    
    metric = better_cos_sim

    @classmethod
    def get_definition(cls, word:str, context:str) -> str:

        definitions = cls._get_all_definitions(word)
        metrics: np.ndarray = cls.metric(context, definitions)

        argmax = metrics.argmax()

        return {
            "definition": definitions[argmax],
            "score"     : float(metrics[argmax])
        }


In [289]:
#  Проверка:

word = "car"
ctx = "I drive my _ with my personnel in it to my dads power plant."
ctx = " ".join(Preprocessor.preprocess(ctx))

BetterLeskWithCosSim.get_definition(word, ctx)

{'definition': 'the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant',
 'score': 0.7972332835197449}

In [290]:
word = "car"
ctx = "I have just bought a new BMW _ from the shop and it's amazing"
ctx = " ".join(Preprocessor.preprocess(ctx))

BetterLeskWithCosSim.get_definition(word, ctx)

{'definition': 'a conveyance for passengers or freight on a cable railway',
 'score': 0.9488925933837891}

Результаты другие. Возможно, изначально метрика была посчитана неверно. В любом случае пока что кажется, что имеется неплохое ускорение. Ниже дублирую код замеров, не забыв при этом записать результаты:

In [273]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)
            context = Preprocessor.preprocess(context)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                context
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

100%|██████████| 10/10 [00:56<00:00,  5.60s/it]


Четырехкратное ускорение, это победа. Отпечатаем результаты:

In [276]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    cos_sim_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, cos_sim_result = result

        jaccard_positive_count += jaccard_result == gold
        cos_sim_positive_count += cos_sim_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim % : {cos_sim_positive_count * 100 / total_count}")

In [277]:
count_results(results)

jaccard % : 56.71641791044776
cos_sim % : 14.925373134328359


Ускорение получили, а вот нормально работать в сделку уже не входило. При чем если с жаккаром такие результаты еще можно понять, то с косинусным подобием точно что-то пошло не так. Попробуем убрать препроц в расчете метрики косинусного подобия:

In [291]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

count_results(results)

100%|██████████| 10/10 [01:20<00:00,  8.06s/it]

jaccard % : 56.71641791044776
cos_sim % : 19.402985074626866


Уже лучше, но все еще плохо. Попробуем вместо максимального значения подобия взять минимальное:

In [296]:
class BetterLeskWithCosSim_InverseArgmax(Lesk):
    
    metric = better_cos_sim

    @classmethod
    def get_definition(cls, word:str, context:str) -> str:

        definitions = cls._get_all_definitions(word)
        metrics: np.ndarray = cls.metric(context, definitions)
        
        #  :o)
        argmax = metrics.argmin()

        return {
            "definition": definitions[argmax],
            "score"     : float(metrics[argmax])
        }

In [293]:
results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                )
            )

count_results(results)

100%|██████████| 10/10 [01:19<00:00,  7.98s/it]

jaccard % : 56.71641791044776
cos_sim % : 43.28358208955224


Результаты неожиданные. Посмотрим, что посчитается на следующих 20 предложениях

In [294]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    cos_sim_positive_count = 0
    inverse_argmax_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, cos_sim_result, inverse_argmax_result = result

        jaccard_positive_count += jaccard_result == gold
        cos_sim_positive_count += cos_sim_result == gold
        inverse_argmax_positive_count += inverse_argmax_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim_normal % : {cos_sim_positive_count * 100 / total_count}")
    print(f"cos_sim_inverse_argmax % : {inverse_argmax_positive_count * 100 / total_count}")

results = list()


for sentence in tqdm(corpus[10:30]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            cos_sim_result = BetterLeskWithCosSim.get_definition(
                word[1],
                " ".join(context)
            )

            inverse_argmax_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    cos_sim_result["definition"],
                    inverse_argmax_result["definition"]
                )
            )

count_results(results)

100%|██████████| 20/20 [06:36<00:00, 19.82s/it]

jaccard % : 54.30107526881721
cos_sim_normal % : 24.193548387096776
cos_sim_inverse_argmax % : 40.32258064516129


Последний тест -- аргмакс с препроцом и без:

In [295]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    inverse_argmax_positive_count_no_preproc = 0
    inverse_argmax_positive_count_w_preproc = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result, inverse_argmax_no_preproc_result, inverse_argmax_w_preproc_result = result

        jaccard_positive_count += jaccard_result == gold
        inverse_argmax_positive_count_no_preproc += inverse_argmax_no_preproc_result == gold
        inverse_argmax_positive_count_w_preproc += inverse_argmax_w_preproc_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")
    print(f"cos_sim_inverse_argmax no preproc % : {inverse_argmax_positive_count_no_preproc * 100 / total_count}")
    print(f"cos_sim_inverse_argmax + preproc % : {inverse_argmax_positive_count_w_preproc * 100 / total_count}")

results = list()


for sentence in tqdm(corpus[:10]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            inverse_argmax_no_preproc_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            inverse_argmax_w_preproc_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(Preprocessor.preprocess(context))
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                    inverse_argmax_no_preproc_result["definition"],
                    inverse_argmax_w_preproc_result["definition"]
                )
            )

count_results(results)

100%|██████████| 10/10 [02:27<00:00, 14.75s/it]

jaccard % : 56.71641791044776
cos_sim_inverse_argmax no preproc % : 43.28358208955224
cos_sim_inverse_argmax + preproc % : 41.791044776119406


Без препроца лучше, дальше весь датасет будем гнать в таком формате: жаккар + препроц, косинусное подобие без препроца. При этом сначала посчитаем жаккара отдельно для всего датасета:

In [301]:
def count_results(results:list) -> None:

    jaccard_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, jaccard_result = result

        jaccard_positive_count += jaccard_result == gold

    print(f"jaccard % : {jaccard_positive_count * 100 / total_count}")



results = list()

for sentence in tqdm(corpus):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            jaccard_result = LeskWithJaccard.get_definition(
                word[1],
                Preprocessor.preprocess(context)
            )

            results.append(
                (
                    gold,
                    jaccard_result["definition"],
                )
            )

count_results(results)

  0%|          | 0/49453 [00:00<?, ?it/s]

100%|██████████| 49453/49453 [05:06<00:00, 161.55it/s]


jaccard % : 43.75961285966163


И теперь посчитаем косинусное подобие отдельно для... 30 *вторых* предложений, потому что мой компьютер не выдержит 140 часов расчетов

In [302]:
def count_results(results:list) -> None:

    cos_sim_positive_count = 0
    total_count = len(results)

    for result in results:
        gold, cos_sim_result = result

        cos_sim_positive_count += cos_sim_result == gold

    print(f"cos_sim % : {cos_sim_positive_count * 100 / total_count}")


results = list()

for sentence in tqdm(corpus[30:60]):
    for i, word in enumerate(sentence):

        if not word[0].strip():  ##  Пропуск однозначного слова
            continue

        else:  ##  Обработка многозначного слова
            gold = wn.lemma_from_key(word[0]).synset().definition()

            context = get_context_mask_target(sentence, i)

            cos_sim_result = BetterLeskWithCosSim_InverseArgmax.get_definition(
                word[1],
                " ".join(context)
            )

            results.append(
                (
                    gold,
                    cos_sim_result["definition"],
                )
            )

count_results(results)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [03:32<00:00,  7.08s/it]

cos_sim % : 47.27272727272727


А тут получилось лучше, чем на жаккаре, мб потому, что выборка маленькая. В любом случае, эксперимент заканчиваю.

# Задание 2 (4 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [304]:
import pandas as pd

In [307]:
df = pd.read_csv(r"./data/train.csv", sep="\t")

In [308]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [309]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [311]:
def get_ARI(grouped_df, cluster):

    ARI = []

    for key, _ in grouped_df:
        # вытаскиваем контексты
        texts = grouped_df.get_group(key)['context'].values

        # создаем пустую матрицу для векторов 
        X = np.zeros((len(texts), 768))

        # переводим тексты в векторы и кладем в матрицу
        for i, text in enumerate(texts):
            X[i] = model.encode(text)

        cluster.fit(X)
        labels = np.array(cluster.labels_)+1 

        # расчитываем метрику для отдельного слова
        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
        
    print(np.mean(ARI)) # усредненная метрика

In [312]:

get_ARI(grouped_df, KMeans(3))

##  7 минут :(



0.09010860802875485


In [ ]:
# # выбираем один из алгоритмов
# # cluster = AffinityPropagation(damping=0.9)
# cluster = KMeans(3)
# #     cluster = DBSCAN(min_samples=1, eps=0.1)